In [ ]:
! pip install transformers
! pip install torch
! pip install datasets evaluate sacrebleu
! pip install sentencepiece
! pip install numpy
! pip install pandas

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB 3.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 4.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.4/106.4 kB 12.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 14.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 16.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 5.9 MB/s eta 0:00:00


In [ ]:
import transformers
from transformers import BertForQuestionAnswering, BertTokenizer
from transformers import BartForConditionalGeneration, BartTokenizer
import torch
from transformers import pipeline
from transformers import MarianMTModel, MarianTokenizer
from datasets import load_dataset, load_metric
import sentencepiece
import re
import asyncio
import textwrap

In [ ]:
### Initialize Content ###
content = """
Mr. Sherlock Holmes, who was usually very late in the mornings, save upon those not infrequent occasions when he was up all night, was seated at the breakfast table. I stood upon the hearth-rug and picked up the stick which our visitor had left behind him the night before. It was a fine, thick piece of wood, bulbous-headed, of the sort which is known as a “Penang lawyer.” Just under the head was a broad silver band nearly an inch across. “To James Mortimer, M.R.C.S., from his friends of the C.C.H.,” was engraved upon it, with the date 1884. It was just such a stick as the old-fashioned family practitioner used to carry—dignified, solid, and reassuring. “How did you know what I was doing? I believe you have eyes in the back of your head.” “I have, at least, a well-polished, silver-plated coffee-pot in front of me,” said he. “But, tell me, Watson, what do you make of our visitor’s stick? Since we have been so unfortunate as to miss him and have no notion of his errand, this accidental souvenir becomes of importance. Let me hear you reconstruct the man by an examination of it.” “ And now, Watson, it only remains for us to find out by wire the identity of the cabman, No. 2704, and then we will drop into one of the Bond Street picture galleries and fill in the time until we are due at the hotel.” “Thank you; I am afraid I cannot claim her acquaintance. We have established a most important fact by these questions, Watson,” Mr. Sherlock Holmes drove with me to the station and gave me his last parting injunctions and advice. “I will not bias your mind by suggesting theories or suspicions, Watson,” said he; “I wish you simply to report facts in the fullest possible manner to me, and you can leave me to do the theorizing.”
I am Stapleton, of Merripit House.” “Your net and box would have told me as much,” said I, “for I knew that Mr. Stapleton was a naturalist. But how did you know me?” “I have been calling on Mortimer, and he pointed you out to me from the window of his surgery as you passed. As our road lay the same way I thought that I would overtake you and introduce myself. I trust that Sir Henry is none the worse for his journey?” “He is very well, thank you.” “We were all rather afraid that after the sad death of Sir Charles the new baronet might refuse to live here. It is asking much of a wealthy man to come down and bury himself in a place of this kind, but I need not tell you that it means a very great deal to the countryside. Sir Henry has, I suppose, no superstitious fears in the matter?” “I do not think that it is likely.” “Of course you know the legend of the fiend dog which haunts the family?” “I have heard it.” “It is extraordinary how credulous the peasants are about here! Any number of them are ready to swear that they have seen such a creature upon the moor.” He spoke with a smile, but I seemed to read in his eyes that he took the matter more seriously. “The story took a great hold upon the imagination of Sir Charles, and I have no doubt that it led to his tragic end.” “But how?” “His nerves were so worked up that the appearance of any dog might have had a fatal effect upon his diseased heart. I fancy that he really did see something of the kind upon that last night in the yew alley. I feared that some disaster might occur, for I was very fond of the old man, and I knew that his heart was weak.”
“It was at my advice that Sir Charles was about to go to London. His heart was, I knew, affected, and the constant anxiety in which he lived, however chimerical the cause of it might be, was evidently having a serious effect upon his health. I thought that a few months among the distractions of town would send him back a new man. Mr. Stapleton, a mutual friend who was much concerned at his state of health, was of the same opinion. At the last instant came this terrible catastrophe. “On the night of Sir Charles’s death Barrymore the butler, who made the discovery, sent Perkins the groom on horseback to me, and as I was sitting up late I was able to reach Baskerville Hall within an hour of the event. I checked and corroborated all the facts which were mentioned at the inquest. I followed the footsteps down the yew alley, I saw the spot at the moor-gate where he seemed to have waited, I remarked the change in the shape of the prints after that point, I noted that there were no other footsteps save those of Barrymore on the soft gravel, and finally I carefully examined the body, which had not been touched until my arrival. Sir Charles lay on his face, his arms out, his fingers dug into the ground, and his features convulsed with some strong emotion to such an extent that I could hardly have sworn to his identity. There was certainly no physical injury of any kind. But one false statement was made by Barrymore at the inquest. He said that there were no traces upon the ground round the body. He did not observe any. But I did—some little distance off, but fresh and clear.” “Footprints?” “Footprints.” “A man’s or a woman’s?” Dr. Mortimer looked strangely at us for an instant, and his voice sank almost to a whisper as he answered. “Mr. Holmes, they were the footprints of a gigantic hound!”
“He could not hope to frighten Sir Henry to death as he had done the old uncle with his bogie hound.” “The beast was savage and half-starved. If its appearance did not frighten its victim to death, at least it would paralyze the resistance which might be offered.” “No doubt. There only remains one difficulty. If Stapleton came into the succession, how could he explain the fact that he, the heir, had been living unannounced under another name so close to the property? How could he claim it without causing suspicion and inquiry?” “It is a formidable difficulty, and I fear that you ask too much when you expect me to solve it. The past and the present are within the field of my inquiry, but what a man may do in the future is a hard question to answer. Mrs. Stapleton has heard her husband discuss the problem on several occasions. There were three possible courses. He might claim the property from South America, establish his identity before the British authorities there and so obtain the fortune without ever coming to England at all, or he might adopt an elaborate disguise during the short time that he need be in London; or, again, he might furnish an accomplice with the proofs and papers, putting him in as heir, and retaining a claim upon some proportion of his income. We cannot doubt from what we know of him that he would have found some way out of the difficulty. And now, my dear Watson, we have had some weeks of severe work, and for one evening, I think, we may turn our thoughts into more pleasant channels.
He meant in the end to have the estate, and he was ready to use any tool or run any risk for that end. His first act was to establish himself as near to his ancestral home as he could, and his second was to cultivate a friendship with Sir Charles Baskerville and with the neighbours. “The baronet himself told him about the family hound, and so prepared the way for his own death. Stapleton, as I will continue to call him, knew that the old man’s heart was weak and that a shock would kill him. So much he had learned from Dr. Mortimer. He had heard also that Sir Charles was superstitious and had taken this grim legend very seriously. His ingenious mind instantly suggested a way by which the baronet could be done to death, and yet it would be hardly possible to bring home the guilt to the real murderer. “Having conceived the idea he proceeded to carry it out with considerable finesse. An ordinary schemer would have been content to work with a savage hound. The use of artificial means to make the creature diabolical was a flash of genius upon his part. The dog he bought in London from Ross and Mangles, the dealers in Fulham Road. It was the strongest and most savage in their possession. He brought it down by the North Devon line and walked a great distance over the moor so as to get it home without exciting any remarks. He had already on his insect hunts learned to penetrate the Grimpen Mire, and so had found a safe hiding-place for the creature. Here he kennelled it and waited his chance.
"""

**PART 1 - Question and Answer Pipeline with Two Models**

In [ ]:
generator1 = pipeline("question-answering") #Default
generator2 = pipeline("question-answering", model="bert-large-uncased-whole-word-masking-finetuned-squad") #Slower Alternative

No model was supplied, defaulted to distilbert-base-cased-distilled-squad and revision 626af31 (https://huggingface.co/distilbert-base-cased-distilled-squad).
Using a pipeline without specifying a model name and revision in production is not recommended.


config.json:   0%|          | 0.00/473 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/261M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

...

model.safetensors:   0%|          | 0.00/1.34G [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-large-uncased-whole-word-masking-finetuned-squad were not used when initializing BertForQuestionAnswering: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [ ]:
### Model 1 - Faster/Less Confident ###
while True:
    question = input("Enter your question (or 'quit' to exit): ")

    if question.lower() == 'quit':
        print("Exiting the program. Goodbye!")
        break

    answer = generator1(question=question, context=content)

    print(f"Answer: '{answer['answer']}' with confidence score {answer['score']}\n")

Enter your question (or 'quit' to exit): Who is the detective?
Answer: 'Mr. Sherlock Holmes' with confidence score 0.6835735440254211

Enter your question (or 'quit' to exit): quit
Exiting the program. Goodbye!


In [ ]:
### Model 2 - Slower/More Confident ###
while True:
    question = input("Enter your question (or 'quit' to exit): ")

    if question.lower() == 'quit':
        print("Exiting the program. Goodbye!")
        break

    answer = generator2(question=question, context=content)

    print(f"Answer: '{answer['answer']}' with confidence score {answer['score']}\n")

Enter your question (or 'quit' to exit): Who is the detective?
Answer: 'Mr. Sherlock Holmes' with confidence score 0.8051921129226685

Enter your question (or 'quit' to exit): Who killed Sir Charles?
Answer: 'Stapleton' with confidence score 0.29721304774284363

Enter your question (or 'quit' to exit): Where was Sir Charles murdered?
Answer: 'the yew alley' with confidence score 0.4079122543334961

Enter your question (or 'quit' to exit): What was the evidence of the case?
Answer: 'footprints of a gigantic hound' with confidence score 0.027300480753183365

Enter your question (or 'quit' to exit): What did Stapleton do?
Answer: 'a naturalist' with confidence score 0.6398656368255615

Enter your question (or 'quit' to exit): What is the detective like?
Answer: 'An ordinary schemer would have been content to work with a savage hound' with confidence score 0.16869644820690155

Enter your question (or 'quit' to exit): Who is the murderer?
Answer: '.' with confidence score 0.2029844671487808

**PART 2 - Translating to Spanish and Back to English**

In [ ]:
translator1 = pipeline("translation", model="Helsinki-NLP/opus-mt-en-es")
retranslator1 = pipeline("translation", model="Helsinki-NLP/opus-mt-es-en")

config.json:   0%|          | 0.00/1.47k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/312M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/293 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/44.0 [00:00<?, ?B/s]

source.spm:   0%|          | 0.00/802k [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/293 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/44.0 [00:00<?, ?B/s]

source.spm:   0%|          | 0.00/826k [00:00<?, ?B/s]

target.spm:   0%|          | 0.00/802k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.59M [00:00<?, ?B/s]

In [ ]:
translator2 = pipeline("translation", model="domenicrosati/opus-mt-en-es-scielo")
retranslator2 = pipeline("translation", model="domenicrosati/opus-mt-es-en-scielo")

config.json:   0%|          | 0.00/1.50k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/310M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/300 [00:00<?, ?B/s]

source.spm:   0%|          | 0.00/802k [00:00<?, ?B/s]

target.spm:   0%|          | 0.00/826k [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/300 [00:00<?, ?B/s]

source.spm:   0%|          | 0.00/826k [00:00<?, ?B/s]

target.spm:   0%|          | 0.00/802k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.72M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/65.0 [00:00<?, ?B/s]

In [ ]:
### Model 1 - More Accurate ###
while True:
    question = input("Enter your question (or 'quit' to exit): ")

    if question.lower() == 'quit':
        print("Exiting the program. Goodbye!")
        break

    answer = generator1(question=question, context=content)

    print(f"Answer: '{answer['answer']}'\n")

    spanish_translation = translator1(answer['answer'])

    print(f"Spanish Answer: '{spanish_translation[0]['translation_text']}'\n")

    retranslated_answer = retranslator1(spanish_translation[0]['translation_text'])

    print(f"Retranslation to English Answer: '{retranslated_answer[0]['translation_text']}'\n")

Enter your question (or 'quit' to exit): Who is the detective?
Answer: 'Mr. Sherlock Holmes'

Spanish Answer: 'Sr. Sherlock Holmes'

Retranslation to English Answer: 'Mr. Sherlock Holmes'

Enter your question (or 'quit' to exit): quit
Exiting the program. Goodbye!


In [ ]:
### Model 2 - Less Accurate ###
while True:
    question = input("Enter your question (or 'quit' to exit): ")

    if question.lower() == 'quit':
        print("Exiting the program. Goodbye!")
        break

    answer = generator1(question=question, context=content)

    print(f"Answer: '{answer['answer']}' with confidence score {answer['score']}\n")

    spanish_translation = translator2(answer['answer'])

    print(f"Spanish Answer: '{spanish_translation[0]['translation_text']}'\n")

    retranslated_answer = retranslator2(spanish_translation[0]['translation_text'])

    print(f"Retranslation to English Answer: '{retranslated_answer[0]['translation_text']}'\n")

Enter your question (or 'quit' to exit): Who is the detective?
Answer: 'Mr. Sherlock Holmes' with confidence score 0.6835735440254211

Spanish Answer: 'Sr. Sherlock Holmes'

Retranslation to English Answer: 'Mr. Sherlock Holmes'

Enter your question (or 'quit' to exit): What is Stapleton like?
Answer: 'naturalist' with confidence score 0.698948860168457

Spanish Answer: 'naturalista'

Retranslation to English Answer: 'naturalist'

Enter your question (or 'quit' to exit): What was it like at Sir Charles' death?
Answer: 'superstitious' with confidence score 0.8960077166557312

Spanish Answer: 'supersticiosa'

Retranslation to English Answer: 'superstitious'

Enter your question (or 'quit' to exit): Did Stapleton have a dog?
Answer: 'It was the strongest and most savage in their possession' with confidence score 0.03838595375418663

Spanish Answer: 'Era el más fuerte y salvaje en su posesión'

Retranslation to English Answer: 'He was the strongest and wildest in his possession'

Enter you

**PART 3 - Summarizing the Sections**

In [ ]:
summarizer1 = pipeline('summarization', model="sshleifer/distilbart-cnn-12-6") #Default
summarizer2 = pipeline('summarization', model="knkarthick/MEETING_SUMMARY")

config.json:   0%|          | 0.00/1.80k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.22G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/337 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

In [ ]:
text1 = """Mr. Sherlock Holmes, who was usually very late in the mornings, save upon those not infrequent occasions when he was up all night, was seated at the breakfast table. I stood upon the hearth-rug and picked up the stick which our visitor had left behind him the night before. It was a fine, thick piece of wood, bulbous-headed, of the sort which is known as a “Penang lawyer.” Just under the head was a broad silver band nearly an inch across. “To James Mortimer, M.R.C.S., from his friends of the C.C.H.,” was engraved upon it, with the date 1884. It was just such a stick as the old-fashioned family practitioner used to carry—dignified, solid, and reassuring. “How did you know what I was doing? I believe you have eyes in the back of your head.” “I have, at least, a well-polished, silver-plated coffee-pot in front of me,” said he. “But, tell me, Watson, what do you make of our visitor’s stick? Since we have been so unfortunate as to miss him and have no notion of his errand, this accidental souvenir becomes of importance. Let me hear you reconstruct the man by an examination of it.” “ And now, Watson, it only remains for us to find out by wire the identity of the cabman, No. 2704, and then we will drop into one of the Bond Street picture galleries and fill in the time until we are due at the hotel.” “Thank you; I am afraid I cannot claim her acquaintance. We have established a most important fact by these questions, Watson,” Mr. Sherlock Holmes drove with me to the station and gave me his last parting injunctions and advice. “I will not bias your mind by suggesting theories or suspicions, Watson,” said he; “I wish you simply to report facts in the fullest possible manner to me, and you can leave me to do the theorizing.”"""
text2 = """I am Stapleton, of Merripit House.” “Your net and box would have told me as much,” said I, “for I knew that Mr. Stapleton was a naturalist. But how did you know me?” “I have been calling on Mortimer, and he pointed you out to me from the window of his surgery as you passed. As our road lay the same way I thought that I would overtake you and introduce myself. I trust that Sir Henry is none the worse for his journey?” “He is very well, thank you.” “We were all rather afraid that after the sad death of Sir Charles the new baronet might refuse to live here. It is asking much of a wealthy man to come down and bury himself in a place of this kind, but I need not tell you that it means a very great deal to the countryside. Sir Henry has, I suppose, no superstitious fears in the matter?” “I do not think that it is likely.” “Of course you know the legend of the fiend dog which haunts the family?” “I have heard it.” “It is extraordinary how credulous the peasants are about here! Any number of them are ready to swear that they have seen such a creature upon the moor.” He spoke with a smile, but I seemed to read in his eyes that he took the matter more seriously. “The story took a great hold upon the imagination of Sir Charles, and I have no doubt that it led to his tragic end.” “But how?” “His nerves were so worked up that the appearance of any dog might have had a fatal effect upon his diseased heart. I fancy that he really did see something of the kind upon that last night in the yew alley. I feared that some disaster might occur, for I was very fond of the old man, and I knew that his heart was weak.”"""
text3 = """“It was at my advice that Sir Charles was about to go to London. His heart was, I knew, affected, and the constant anxiety in which he lived, however chimerical the cause of it might be, was evidently having a serious effect upon his health. I thought that a few months among the distractions of town would send him back a new man. Mr. Stapleton, a mutual friend who was much concerned at his state of health, was of the same opinion. At the last instant came this terrible catastrophe. “On the night of Sir Charles’s death Barrymore the butler, who made the discovery, sent Perkins the groom on horseback to me, and as I was sitting up late I was able to reach Baskerville Hall within an hour of the event. I checked and corroborated all the facts which were mentioned at the inquest. I followed the footsteps down the yew alley, I saw the spot at the moor-gate where he seemed to have waited, I remarked the change in the shape of the prints after that point, I noted that there were no other footsteps save those of Barrymore on the soft gravel, and finally I carefully examined the body, which had not been touched until my arrival. Sir Charles lay on his face, his arms out, his fingers dug into the ground, and his features convulsed with some strong emotion to such an extent that I could hardly have sworn to his identity. There was certainly no physical injury of any kind. But one false statement was made by Barrymore at the inquest. He said that there were no traces upon the ground round the body. He did not observe any. But I did—some little distance off, but fresh and clear.” “Footprints?” “Footprints.” “A man’s or a woman’s?” Dr. Mortimer looked strangely at us for an instant, and his voice sank almost to a whisper as he answered. “Mr. Holmes, they were the footprints of a gigantic hound!”"""
text4 = """“He could not hope to frighten Sir Henry to death as he had done the old uncle with his bogie hound.” “The beast was savage and half-starved. If its appearance did not frighten its victim to death, at least it would paralyze the resistance which might be offered.” “No doubt. There only remains one difficulty. If Stapleton came into the succession, how could he explain the fact that he, the heir, had been living unannounced under another name so close to the property? How could he claim it without causing suspicion and inquiry?” “It is a formidable difficulty, and I fear that you ask too much when you expect me to solve it. The past and the present are within the field of my inquiry, but what a man may do in the future is a hard question to answer. Mrs. Stapleton has heard her husband discuss the problem on several occasions. There were three possible courses. He might claim the property from South America, establish his identity before the British authorities there and so obtain the fortune without ever coming to England at all, or he might adopt an elaborate disguise during the short time that he need be in London; or, again, he might furnish an accomplice with the proofs and papers, putting him in as heir, and retaining a claim upon some proportion of his income. We cannot doubt from what we know of him that he would have found some way out of the difficulty. And now, my dear Watson, we have had some weeks of severe work, and for one evening, I think, we may turn our thoughts into more pleasant channels."""
text5 = """He meant in the end to have the estate, and he was ready to use any tool or run any risk for that end. His first act was to establish himself as near to his ancestral home as he could, and his second was to cultivate a friendship with Sir Charles Baskerville and with the neighbours. “The baronet himself told him about the family hound, and so prepared the way for his own death. Stapleton, as I will continue to call him, knew that the old man’s heart was weak and that a shock would kill him. So much he had learned from Dr. Mortimer. He had heard also that Sir Charles was superstitious and had taken this grim legend very seriously. His ingenious mind instantly suggested a way by which the baronet could be done to death, and yet it would be hardly possible to bring home the guilt to the real murderer. “Having conceived the idea he proceeded to carry it out with considerable finesse. An ordinary schemer would have been content to work with a savage hound. The use of artificial means to make the creature diabolical was a flash of genius upon his part. The dog he bought in London from Ross and Mangles, the dealers in Fulham Road. It was the strongest and most savage in their possession. He brought it down by the North Devon line and walked a great distance over the moor so as to get it home without exciting any remarks. He had already on his insect hunts learned to penetrate the Grimpen Mire, and so had found a safe hiding-place for the creature. Here he kennelled it and waited his chance."""

In [ ]:
# Model 1
print("Section 1 Summary: ")
print(textwrap.fill(summarizer1(text1)[0]['summary_text'], width=150))
print()
print("Section 2 Summary: ")
print(textwrap.fill(summarizer1(text2)[0]['summary_text'], width=150))
print()
print("Section 3 Summary: ")
print(textwrap.fill(summarizer1(text3)[0]['summary_text'], width=150))
print()
print("Section 4 Summary: ")
print(textwrap.fill(summarizer1(text4)[0]['summary_text'], width=150))
print()
print("Section 5 Summary: ")
print(textwrap.fill(summarizer1(text5)[0]['summary_text'], width=150))

Section 1 Summary: 
 Mr. Sherlock Holmes, who was usually very late in the mornings, was seated at the breakfast table . I stood upon the hearth-rug and picked up the
stick which our visitor had left behind him the night before . It was a fine, thick piece of wood, bulbous-headed, of the sort which is known as a
“Penang lawyer” Just under the head was a broad silver band nearly an inch across . “To James Mortimer, M.R.C.S., from his friends of the C.C.,” was
engraved upon it, with the date 1884 .

Section 2 Summary: 


KeyboardInterrupt: ignored

In [ ]:
# Model 2
print("Section 1 Summary: ")
print(textwrap.fill(summarizer2(text1)[0]['summary_text'], width=150))
print()
print("Section 2 Summary: ")
print(textwrap.fill(summarizer2(text2)[0]['summary_text'], width=150))
print()
print("Section 3 Summary: ")
print(textwrap.fill(summarizer2(text3)[0]['summary_text'], width=150))
print()
print("Section 4 Summary: ")
print(textwrap.fill(summarizer2(text4)[0]['summary_text'], width=150))
print()
print("Section 5 Summary: ")
print(textwrap.fill(summarizer2(text5)[0]['summary_text'], width=150))

Section 1 Summary: 
Sherlock Holmes found a piece of wood stick with a silver band attached to it belonging to James Mortimer, M.R.C.H. engraved upon it, with the date
1884, at the breakfast table. Sherlock Holmes drove to the station and gave me his last parting

Section 2 Summary: 
After the death of Sir Charles the new baronet decided to move to Merripit House. He was a naturalist and a great fan of the legend of the fiend dog.

Section 3 Summary: 
On the night of Sir Charles’s death Barrymore the butler, who made the discovery, sent Perkins the groom on horseback to me. I arrived at Baskerville
Hall within an hour of the event. I followed the footsteps down the yew alley, I saw the

Section 4 Summary: 
According to Mrs. Stapleton, the heir to the property would have to come to England to claim it. She has heard her husband discuss the problem and
suggests three possible solutions: he might claim the property from South America, establish his identity before the British authorities 